In [3]:
import os#
from pathlib import Path#

import pandas as pd
import torch


import numpy as np
from stk_search.utils.config_utils import read_config, save_config
from stk_search.utils import update_datasets
import importlib
importlib.reload(update_datasets)
os.makedirs('search_experiment/search_exp_database', exist_ok=True)

In [4]:
# load Search space and check conditions
import pickle
search_space_loc = "search_space/search_space_test.pkl"


SP = pickle.load(open(search_space_loc, 'rb'))
SP.update()
print(SP.syntax)
print(SP.conditions_list)
print(f"size of the search space is : {SP.get_space_size():.2e}")
SP.df_precursors.head()

[0, 1, 2, 3, 4, 5]
[[], [], [], [], [], []]
size of the search space is : 0.00e+00


,electron affinity (eV),homo lumo_gap (eV),ionisation potential (eV),total energy (au),HOMO-LUMO GAP (eV),ES1,fosc1,InChIKey,mol_opt,Atom_num
0,3.0685,0.095166,9.1140,-56.555859,1.672081,3.787,0.0003,GVYASUPVXDSGHN-UQCOIBPSSA-N,<rdkit.Chem.rdchem.Mol object at 0x7a1624459130>,22
1,2.8466,0.038435,9.3325,-56.536670,2.258122,3.514,0.0016,HNRHIGPDSSXEJA-FARCUNLSSA-N,<rdkit.Chem.rdchem.Mol object at 0x7a16244590e0>,22
2,2.9365,0.072094,9.2252,-56.533763,2.009171,3.414,0.0003,SNMJIXVGAOBXCS-UHFFFAOYSA-N,<rdkit.Chem.rdchem.Mol object at 0x7a1624459180>,22
3,2.5901,0.064465,8.9842,-48.098467,2.153852,3.364,0.0008,LWMBLJLODRWDDF-NYYWCZLTSA-N,<rdkit.Chem.rdchem.Mol object at 0x7a16244591d0>,22
4,2.5596,0.076792,8.8405,-48.091982,1.967541,3.342,0.0004,BJFIUKSBVAJFOJ-WTKPLQERSA-N,<rdkit.Chem.rdchem.Mol object at 0x7a1624459220>,22


In [5]:
from stk_search.Objective_function import  Look_up_table
df_total_path = "data_example/Molecule_database/30K_benchmark_150524.csv"
df_total = pd.read_csv(df_total_path)
oligomer_size = 6
target_name = 'target'
aim = 'maximise'
df_total['target'] = (
        -np.abs(df_total["ES1"] - 3)
        - np.abs(df_total["ionisation potential (eV)"] - 5.5)
        + np.log10(df_total["fosc1"])
    )
df_total.to_csv(df_total_path, index=False)
# define the evaluation function
objective_function = Look_up_table(
            df_total, oligomer_size, target_name=target_name, aim=aim
        )

In [6]:
# test evaluation function
molecule_id = 0
molecule_properties = SP.check_df_for_element_from_sp(df_total).iloc[[molecule_id]]
print(molecule_properties['InChIKey'])
molecule_properties[[f'InChIKey_{x}' for x in range(oligomer_size)]]
objective_function.evaluate_element(molecule_properties[[f'InChIKey_{x}' for x in range(oligomer_size)]])

0    CLUOOVKTWZALSH-UHFFFAOYSA-N
Name: InChIKey, dtype: object


(-2.5070288713583553, 'CLUOOVKTWZALSH-UHFFFAOYSA-N')

In [7]:
# initialise search algorithm 
from stk_search.Search_algorithm import BayesianOptimisation
from stk_search.Search_algorithm import Ea_surrogate

from stk_search.Search_algorithm.Search_algorithm import evolution_algorithm
from stk_search.Search_algorithm import Search_algorithm
importlib.reload(Search_algorithm)
importlib.reload(Ea_surrogate)
importlib.reload(BayesianOptimisation)
which_acquisition = 'EI'
lim_counter = 10
BO_learned = BayesianOptimisation.BayesianOptimisation(which_acquisition=which_acquisition, lim_counter=lim_counter)
EA = Search_algorithm.evolution_algorithm()
SUEA = Ea_surrogate.Ea_surrogate()
BO_Mord = BayesianOptimisation.BayesianOptimisation(which_acquisition=which_acquisition, lim_counter=lim_counter)

In [12]:
# load the representation and the model 
from stk_search.Search_algorithm import Ea_surrogate

from stk_search.Representation import  Representation_poly_3d
import importlib
importlib.reload(Representation_poly_3d)
from stk_search.geom3d import pl_model
import torch.nn.functional as Functional
from stk_search.geom3d import train_models
from stk_search.Representation import  Representation_poly_3d

import importlib
importlib.reload(Representation_poly_3d)
importlib.reload(Ea_surrogate)


#config_dir = "/rds/general/user/ma11115/home/STK_Search/STK_search/data/representation_learning/6-frag/target/240430/SchNet/splitrand-nummol68864/Emb_dim_12"
config_dir = "data_example/representation_learning/splitrand-nummol20000/"
config, min_val_loss = train_models.get_best_embedding_model(config_dir)

SUEA = Ea_surrogate.Ea_surrogate()
SUEA.config_dir = config_dir
SUEA.load_representation_model()
BO_learned.config_dir = config_dir
BO_learned.load_representation_model()


Model loaded:  SchNet
Model loaded:  SchNet


(<stk_search.Representation.Representation_poly_3d.Representation_poly_3d at 0x7a160f89ce20>,
 Pymodel_new(
   (molecule_3D_repr): SchNet(hidden_channels=128, num_filters=128, num_interactions=8, num_gaussians=51, cutoff=6)
   (graph_pred_linear): Linear(in_features=128, out_features=1, bias=True)
   (transform_to_opt): Linear(in_features=128, out_features=128, bias=True)
 ))

In [13]:
# initialise the representation of the molecules used
# here we use the dataframe of the precursors that has PCA of desrciprtors calculated using mordred
from stk_search.Representation import (
    Representation_from_fragment,
    Representation_3d_from_fragment,
)

df_representation_path = "data_example/precursor/df_mordred_24072024.pkl"
df_representation = pd.read_pickle(df_representation_path)
df_representation.dropna(axis=1, inplace=True)
# check if the representation dataframe contains all the elements in teh search space
print(
    "number of element not in the representation dataframe ",
    -(SP.df_precursors["InChIKey"].isin(df_representation["InChIKey"]).sum()
    - SP.df_precursors.shape[0]),
)
if -(SP.df_precursors["InChIKey"].isin(df_representation["InChIKey"]).sum() - SP.df_precursors.shape[0]) > 0:
    SP.df_precursors = SP.df_precursors[SP.df_precursors["InChIKey"].isin(df_representation["InChIKey"])]
    with open(search_space_loc, "wb") as f:
        pickle.dump(SP, f)
frag_properties = df_representation.select_dtypes(include=[np.number]).columns
print(frag_properties)
BO_Mord.Representation = Representation_from_fragment.Representation_from_fragment(
    df_representation, frag_properties
)


number of element not in the representation dataframe  0
Index(['PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7',
       'PCA_8', 'PCA_9'],
      dtype='object')


In [15]:
# test representation
import swifter
molecule_id = 0
molecule_properties = SP.check_df_for_element_from_sp(df_total).iloc[[molecule_id]]
print(molecule_properties['InChIKey'])
X_rpr = SUEA.Representation.generate_repr(molecule_properties[[f'InChIKey_{x}' for x in range(oligomer_size)]])
print(X_rpr)
X_rpr = BO_learned.Representation.generate_repr(molecule_properties[[f'InChIKey_{x}' for x in range(oligomer_size)]])
print(X_rpr)

0    CLUOOVKTWZALSH-UHFFFAOYSA-N
Name: InChIKey, dtype: object


Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

KeyError: 'PNLCAIFEPYYUCC-UHFFFAOYSA-N'

In [ ]:
from stk_search import SearchExp

importlib.reload(SearchExp)


# set experiment conditions
def define_and_run_search(
    search_algorithm=EA,
    num_elem_initialisation=50,
    number_of_iterations=100,
    verbose=True,
    case_name="test",
):

    S_exp = SearchExp.SearchExp(
        SP,
        search_algorithm,
        objective_function,
        number_of_iterations,
        verbose=verbose,
    )
    benchmark = True
    S_exp.output_folder = (
        f"output/search_experiment/{oligomer_size}_frag/" + case_name
    )
    S_exp.num_elem_initialisation = num_elem_initialisation
    S_exp.benchmark = benchmark
    S_exp.df_total = df_total
    # Save search inputs
    import json
    import subprocess

    def save_run_search_inputs(inputs, save_path="run_search_new_inputs.json"):
        # Get the current git version
        git_version = (
            subprocess.check_output(["git", "rev-parse", "HEAD"])
            .strip()
            .decode("utf-8")
        )

        # Add the git version to the inputs
        inputs["git_version"] = git_version

        # Save the inputs to a file
        with open(save_path, "w") as f:
            json.dump(inputs, f)

        print("Inputs saved.")

    input_json = {}
    input_json["search_space_loc"] = search_space_loc
    input_json["search_algorithm"] = search_algorithm.name
    input_json["objective_function"] = objective_function.__str__()
    input_json["number_of_iterations"] = number_of_iterations
    input_json["verbose"] = verbose
    input_json["num_elem_initialisation"] = num_elem_initialisation
    input_json["benchmark"] = benchmark
    input_json["df_total"] = df_total_path
    input_json["df_representation"] = df_representation_path
    if 'Representation' in search_algorithm.__dict__.keys(): 
        input_json["representation"] = search_algorithm.Representation.name
    else:
        input_json["representation"] = None
    input_json["frag_properties"] = list(frag_properties)
    input_json["which_acquisition"] = which_acquisition
    input_json["run_search_name"] = S_exp.search_exp_name
    input_json["search_output_folder"] = S_exp.output_folder
    input_json["date"] = S_exp.date
    input_json["oligomer_size"] = oligomer_size
    input_json["config_dir"] = config_dir
    input_json["case_name"] = case_name
    save_path = f"output/search_experiment/search_exp_database/{S_exp.search_exp_name}.json"
    os.makedirs(
        "output/search_experiment/search_exp_database", exist_ok=True
    )
    save_run_search_inputs(input_json, save_path)
    S_exp.run_seach()
    return max(S_exp.ids_acquired)

In [ ]:
for _ in range(10):
    define_and_run_search(search_algorithm=EA,case_name="EA")

Inputs saved.
Inputs saved.
Inputs saved.
Inputs saved.
Inputs saved.
Inputs saved.
Inputs saved.
Inputs saved.
Inputs saved.
Inputs saved.


In [ ]:
for _ in range(10):
    define_and_run_search(search_algorithm=SUEA,case_name="SUEA")

Inputs saved.


Pandas Apply:   0%|          | 0/99 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/38 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/129 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/17 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/103 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/124 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/3 [00:00<?, ?it/s]

KeyError: '[143] not in index'

In [ ]:
for _ in range(5):
    define_and_run_search(search_algorithm=BO_learned,case_name="BO_learned")

Inputs saved.


Pandas Apply:   0%|          | 0/50 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/87 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/37 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/239 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/242 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/254 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/8 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/87 [00:00<?, ?it/s]

In [ ]:
for _ in range(5):
    define_and_run_search(search_algorithm=BO_Mord,case_name="BO_Mord")

Inputs saved.


/home/mohammed/anaconda3/envs/stk_search2/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
/home/mohammed/anaconda3/envs/stk_search2/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(
/home/mohammed/anaconda3/envs/stk_search2/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/mohammed/anaconda3/envs/stk_search2/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/mohammed/anaconda3/envs/stk_search2/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/home/mohammed/anaconda3/

149

# load results and plot some metrics

In [ ]:

import glob
import pickle
import json
def load_search_list(df):
    search_list = []
    for index, row in df.iterrows():
        search_list.append(pickle.load(open(row['search_exp_file'], "rb")))
    return search_list
save_path = f"output/search_experiment/search_exp_database"
json_files = glob.glob(f"{save_path}/*.json")
list_json = []
for json_file in json_files:
    try:
        with open(json_file, 'r') as f:
            list_json.append(json.load(f))
        f.close()
    except: 
        print('error oppening',json_file)
df = pd.DataFrame(list_json)
df['search_exp_file'] = df['search_output_folder']+"/"+df['date']+"/results_"+df['run_search_name']+'.pkl'
print(df.shape)
df_all = df.copy()
df_all.head()


(32, 20)


,search_space_loc,search_algorithm,objective_function,number_of_iterations,verbose,num_elem_initialisation,benchmark,df_total,df_representation,representation,frag_properties,which_acquisition,run_search_name,search_output_folder,date,oligomer_size,config_dir,case_name,git_version,search_exp_file
0,search_space/search_space_test.pkl,Bayesian_Optimisation,<stk_search.Objective_function.Look_up_table o...,100,True,50,True,data_example/Molecule_database/30K_benchmark_1...,data_example/precursor/df_mordred_24072024.pkl,Representation_poly_3d,"[PCA_0, PCA_1, PCA_2, PCA_3, PCA_4, PCA_5, PCA...",EI,3246270be1a3453090bddfad891001a6,output/search_experiment/6_frag/BO_learned,20240802,6,data_example/representation_learning/splitrand...,BO_learned,ab9492b1a0b17402e63f34abb4f1a5529ff4196c,output/search_experiment/6_frag/BO_learned/202...
1,search_space/search_space_test.pkl,Evolution_algorithm,<stk_search.Objective_function.Look_up_table o...,100,True,50,True,data_example/Molecule_database/30K_benchmark_1...,data_example/precursor/df_mordred_24072024.pkl,None,"[PCA_0, PCA_1, PCA_2, PCA_3, PCA_4, PCA_5, PCA...",EI,f8114b25c4834dfebbf397fa962b786c,output/search_experiment/6_frag/EA,20240802,6,data_example/representation_learning/splitrand...,EA,ab9492b1a0b17402e63f34abb4f1a5529ff4196c,output/search_experiment/6_frag/EA/20240802/re...
2,search_space/search_space_test.pkl,Evolution_algorithm,<stk_search.Objective_function.Look_up_table o...,100,True,50,True,data_example/Molecule_database/30K_benchmark_1...,data_example/precursor/df_mordred_24072024.pkl,None,"[PCA_0, PCA_1, PCA_2, PCA_3, PCA_4, PCA_5, PCA...",EI,396f0f5c169545e5b7e0fa1b9312fe39,output/search_experiment/6_frag/EA,20240802,6,data_example/representation_learning/splitrand...,EA,ab9492b1a0b17402e63f34abb4f1a5529ff4196c,output/search_experiment/6_frag/EA/20240802/re...
3,search_space/search_space_test.pkl,Evolution_algorithm,<stk_search.Objective_function.Look_up_table o...,100,True,50,True,data_example/Molecule_database/30K_benchmark_1...,data_example/precursor/df_mordred_24072024.pkl,None,"[PCA_0, PCA_1, PCA_2, PCA_3, PCA_4, PCA_5, PCA...",EI,ff78b9b8c82540a6b4b543b68ab138b6,output/search_experiment/6_frag/EA,20240802,6,data_example/representation_learning/splitrand...,EA,ab9492b1a0b17402e63f34abb4f1a5529ff4196c,output/search_experiment/6_frag/EA/20240802/re...
4,search_space/search_space_test.pkl,Surrogate_EA,<stk_search.Objective_function.Look_up_table o...,100,True,50,True,data_example/Molecule_database/30K_benchmark_1...,data_example/precursor/df_mordred_24072024.pkl,Representation_poly_3d,"[PCA_0, PCA_1, PCA_2, PCA_3, PCA_4, PCA_5, PCA...",EI,c8b89fe2b3414f7a8ca818cc90b2c95b,output/search_experiment/6_frag/SUEA,20240802,6,data_example/representation_learning/splitrand...,SUEA,ab9492b1a0b17402e63f34abb4f1a5529ff4196c,output/search_experiment/6_frag/SUEA/20240802/...


In [ ]:
num_iteration = 20
num_elem_initialisation = 50
oligomer_size = 6 
#df_filtered = df_all[df_all['num_iteration']==num_iteration]
#df_filtered = df_filtered[df_filtered['num_elem_initialisation']==num_elem_initialisation]
df_filtered = df_all[df_all['oligomer_size']==oligomer_size]
print(df_filtered.shape)
df_list = df_filtered.groupby(['which_acquisition','config_dir','search_algorithm','case_name'])
print(len(df_list))
df_list.describe()

(32, 20)
4


number_of_iterations  \
                                                                                                                     count   
which_acquisition config_dir                                         search_algorithm      case_name                         
EI                data_example/representation_learning/splitrand-... Bayesian_Optimisation BO_Mord                     2.0   
                                                                                           BO_learned                  3.0   
                                                                     Evolution_algorithm   EA                         16.0   
                                                                     Surrogate_EA          SUEA                       11.0   

                                                                                                                  \
                                                                                                            mean   
which_acquisition config_dir                                         search_algorithm      case_name               
EI                data_example/representation_learning/splitrand-... Bayesian_Optimisation BO_Mord     60.000000   
                                                                                           BO_learned  73.333333   
                                                                     Evolution_algorithm   EA          95.000000   
                                                                     Surrogate_EA          SUEA        92.727273   

                                                                                                                  \
                                                                                                             std   
which_acquisition config_dir                                         search_algorithm      case_name               
EI                data_example/representation_learning/splitrand-... Bayesian_Optimisation BO_Mord     56.568542   
                                                                                           BO_learned  46.188022   
                                                                     Evolution_algorithm   EA          20.000000   
                                                                     Surrogate_EA          SUEA        24.120908   

                                                                                                             \
                                                                                                        min   
which_acquisition config_dir                                         search_algorithm      case_name          
EI                data_example/representation_learning/splitrand-... Bayesian_Optimisation BO_Mord     20.0   
                                                                                           BO_learned  20.0   
                                                                     Evolution_algorithm   EA          20.0   
                                                                     Surrogate_EA          SUEA        20.0   

                                                                                                              \
                                                                                                         25%   
which_acquisition config_dir                                         search_algorithm      case_name           
EI                data_example/representation_learning/splitrand-... Bayesian_Optimisation BO_Mord      40.0   
                                                                                           BO_learned   60.0   
                                                                     Evolution_algorithm   EA          100.0   
                                                                     Surrogate_EA          SUEA        100.0   

                              

In [ ]:
import pandas as pd
def join_name(x):
    return '_'.join(x)

def load_search_list(df,min_num_iteration):
    search_list = []
    for index, row in df.iterrows():
        if min_num_iteration <= row['max_num_eval']:
            search_list.append(pd.read_pickle(row['search_exp_file']))
    return search_list
def get_results_length(x):
    try:
        return len(pd.read_pickle(x)['fitness_acquired'])
    except Exception as e:
        #print(e)
        return 0
results_dict = {}
num_searches_with_min, df_plot_dict = {}, {}
min_num_iteration = 100
for name , df in df_list:
    if len(df) != 0: 
        df['max_num_eval']=df['search_exp_file'].apply(lambda x: get_results_length(x))
        key = join_name(name)
        df['key'] = key
        if df['max_num_eval'].max() >= min_num_iteration:
            results_dict[key] = load_search_list(df,min_num_iteration)
            num_searches_with_min[key] = (len(results_dict[key]), df['df_total'].values[0])
df_summary_1 = pd.DataFrame.from_dict(num_searches_with_min, orient='index',columns=['number of explorations with min iterations','df_path'])
#df_summary = df_summary.merge(df_summary_1, left_index=True, right_index=True)
df_summary_1['key'] = df_summary_1.index    
df_summary_1['case_name'] = df_summary_1['key'].apply(lambda x: x.split('_')[-2]+'_'+x.split('_')[-1])
df_summary_1['num_mol_training'] = df_summary_1['key'].apply(lambda x: x.split('nummol')[-1])
df_summary_1['wide']=df_summary_1['key'].apply(lambda x: '_wide' in x)
df_summary_1['narrow']=df_summary_1['key'].apply(lambda x: '_narrow' in x)
df_summary_1['total']=df_summary_1['key'].apply(lambda x: '_total' in x)
#df_summary_1['key'] = df_summary_1['case'] 
df_summary_1.reset_index(inplace=True)
df_summary_1

,index,number of explorations with min iterations,df_path,key,case_name,num_mol_training,wide,narrow,total
0,EI_data_example/representation_learning/splitr...,1,data_example/Molecule_database/30K_benchmark_1...,EI_data_example/representation_learning/splitr...,BO_Mord,20000/_Bayesian_Optimisation_BO_Mord,False,False,False
1,EI_data_example/representation_learning/splitr...,2,data_example/Molecule_database/30K_benchmark_1...,EI_data_example/representation_learning/splitr...,BO_learned,20000/_Bayesian_Optimisation_BO_learned,False,False,False
2,EI_data_example/representation_learning/splitr...,15,data_example/Molecule_database/30K_benchmark_1...,EI_data_example/representation_learning/splitr...,algorithm_EA,20000/_Evolution_algorithm_EA,False,False,False
3,EI_data_example/representation_learning/splitr...,10,data_example/Molecule_database/30K_benchmark_1...,EI_data_example/representation_learning/splitr...,EA_SUEA,20000/_Surrogate_EA_SUEA,False,False,False


In [ ]:
import datetime
import pickle
from stk_search.utils import Search_results_plot
from stk_search.utils import plot_results_all
import importlib
importlib.reload(Search_results_plot)
importlib.reload(plot_results_all)
save_folder = 'data/figures/'+datetime.datetime.now().strftime("_%Y_%m_%d")
plot_function_list_single = [Search_results_plot.plot_simple_regret,Search_results_plot.plot_inst_regret,Search_results_plot.plot_cumulative_regret,
                      Search_results_plot.plot_number_of_molecule_discovered,Search_results_plot.plot_rate_of_discovery,]
plot_function_list_multi = [Search_results_plot.plot_simple_regret_batch,Search_results_plot.plot_number_of_molecule_discovered_sum,Search_results_plot.plot_total_rate_of_discovery, ]

In [10]:
%matplotlib inline
df_plot = df_summary_1.copy()
df_plot = df_plot.loc[[0,1,2,3]]
fig, axes,metric_dict_res = plot_results_all.plot_metric(df_plot,plot_function_list_single,results_dict,df_list,nb_iterations=120)
fig.tight_layout


NameError: name 'df_summary_1' is not defined